In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import linalg
import math
from neicio.readstation import readStation
from neicio.shake import ShakeGrid
from neicio.gmt import GMTGrid
import time
from openquake.hazardlib.imt import from_string
from openquake.hazardlib.site import Site, SiteCollection
from openquake.hazardlib.geo import Point
from openquake.hazardlib.correlation import JB2009CorrelationModel
from openquake.hazardlib.correlation import BaseCorrelationModel
from matplotlib import cm
import sys
sys.path.append('/Users/sverros/Documents/Modules/')
from Correlation.setup import initialize
from Correlation.loop_rand import main
from Correlation.realizations import realizations

In [2]:
# Variable of interest                                                                                                                                                                                                        
voi = 'PGA'

# Intensity Factor
intensity_factor = 1

vscorr = True

# Get shakemap for desired variable, PGA, uncertainty grid and stationdata                                                                                                                                                    
# Selected Stations: Units in pctg                                                                                                                                                                                            
shakemap = ShakeGrid('Inputs/grid.xml', variable = '%s' % voi)

# Uncertainty Data: Units in ln(pctg)                                                                                                                                                                                         
uncertainty = ShakeGrid('Inputs/uncertainty.xml', variable= 'STD%s' % (voi))

# Station Data: Units in pctg                                                                                                                                                                                                 
stationlist = 'Inputs/stationlist.xml'
stationdata = readStation(stationlist)

print np.size(stationdata['lon']), 'stations'

# Sets up grid spacing, site collections, and other data values
# optional parameters: dm, dn, the grid discritization defaulting to 1
variables = initialize(shakemap, uncertainty, stationdata, vscorr)
M = variables['M']
N = variables['N']
K = variables['K']

# Calculate the random array, stored for testing
#rand = np.random.randn(M*N).flatten()
rand = []

with open('rand2.txt') as f:
    for line in f:
        rand.append(float(line))
f.close()



0 stations
	Initialization Time: 0.26093506813


In [ ]:
R = [15, 25, 35]

f = open('workfileNR26_uncorr.txt', 'w')

f.write('Northridge, %d stations, max_R = %d\n' %(np.size(stationdata['pga']), R[-1]))
s = str(np.size(R))
f.write(s+'\n')
s = str(M)
f.write(s+'\n')
s = str(N)
f.write(s+'\n')

data_NEW = np.zeros([M*N, np.size(R)])

for k in range(0,np.size(R)):
    out = main(variables, R[k], voi, rand, vscorr, 0.9)
    data_NEW[:,k] = np.reshape(out['cor'], [M*N])
    
    for j in range(0, M*N):
        s = str(data_NEW[j,k])
        f.write(s+'\n')
                         
f.close()

Finishing step: 0
Finishing step: 5000
Finishing step: 10000
Finishing step: 15000
Finishing step: 20000
Finishing step: 25000
Finishing step: 30000
[[ 0.43952639  0.8201699   0.59176983 ..., -0.64209665 -0.27433631 -0.73231265]
 [ 0.78701277  0.86101106  0.47136681 ..., -0.56172336 -0.98389822 -0.59573979]
 [ 0.62299911  0.63758312  0.3290209  ..., -0.7958823  -0.96341637 -0.32712981]
 ..., 
 [-0.21628126  0.03512448 -0.11203668 ..., -0.13019402  0.07165379 -0.08995787]
 [ 0.2208704  -0.06373164 -0.1650547  ..., -0.09311011 -0.41524354 -0.29866398]
 [ 0.09641299  0.46229572  0.3198288  ...,  0.08113342  0.27062742  0.0977373 ]]
Finishing step: 0
Finishing step: 5000
Finishing step: 10000


In [12]:
!cp 'workfileNR26_uncorr.txt' '/Users/sverros/Documents/Testing/radius_NR0_rand3_uncorrelated.txt'